In [11]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.linear_model
import sklearn.tree
import sklearn.ensemble
import sklearn.naive_bayes
import sklearn.neural_network
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [12]:
#Dataset 2 : Default of credit card clients
data = np.loadtxt('default_of_credit_card_clients.csv', delimiter=',',skiprows=2)
X_train, X_test, y_train, y_test = train_test_split(
    data[:,1:24],data[:,24], test_size=0.2, random_state=0)

In [13]:
#k-Nearest neighbours classification
knn_model = sklearn.neighbors.KNeighborsClassifier(n_neighbors=2)
knn_model.fit(X_train, y_train)
y_pred = knn_model.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.7656666666666667

In [14]:
#Logistic regression (for classification)
logistic_model = sklearn.linear_model.LogisticRegression(fit_intercept=True, penalty='l2', solver='lbfgs',max_iter = 10000)
logistic_model.fit(X_train, y_train);
y_pred = logistic_model.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.784

In [15]:
#Decision tree classification
DTC_model = sklearn.tree.DecisionTreeClassifier()
DTC_model.fit(X_train, y_train)
y_pred = DTC_model.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.7345

In [16]:
#Random forest classification
RFC_model = sklearn.ensemble.RandomForestClassifier()
RFC_model.fit(X_train, y_train)
y_pred = RFC_model.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

/home/kbeepi/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8076666666666666

In [17]:
#AdaBoost classification
ABC_model = sklearn.ensemble.AdaBoostClassifier()
ABC_model.fit(X_train, y_train)
y_pred = ABC_model.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.826

In [18]:
#Gaussian naive Bayes classification
GNB_model = sklearn.naive_bayes.GaussianNB()
GNB_model.fit(X_train, y_train)
y_pred = ABC_model.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.826

In [19]:
#Neural network classification
NNC_model = sklearn.neural_network.MLPClassifier()
NNC_model.fit(X_train, y_train)
y_pred = NNC_model.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.6063333333333333

In [20]:
#SVM classifier
svm_model = sklearn.svm.SVC()
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

/home/kbeepi/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.7841666666666667